# Walkthrough using napari
From raw image to plot the area through time of the wound

In [1]:
import napari
from magicgui import magicgui
from napari.types import ImageData, LabelsData, LayerDataTuple
from magicgui import widgets
from magicgui.tqdm import trange
from base_segmentation import Otsu, YAPIC, ChanVese
from utils import load_params, load_tiff, save_tiff
from func_segmentation import get_holes_mask
from pathlib import Path
import numpy as np

In [2]:
viewer = napari.Viewer()

In [5]:
# load all Segmentation Parameters

param_path = Path('../parameters/default_segmentation_workflow.yml')
params = load_params(param_path)

NameError: name 'Path' is not defined

In [10]:

import pathlib

@magicgui(
    call_button='Load tif image'
)
def image_loader(
    filename=pathlib.Path(params['input']['path'])
) -> ImageData:
    im_axes = params['input']['axes']
    im = load_tiff(filename, im_axes)
    return im
# add our new magicgui widget to the viewer
viewer.window.add_dock_widget(image_loader)

: 

In [5]:
# viz utils function 

def get_container(container_name, params):

    table = widgets.Table(value=params)
    btn = widgets.PushButton(text="Edit Params")

    @btn.changed.connect
    def show_params(value: bool):
        table.show()

    wdg_segment = [widgets.CheckBox(label=""), btn]
    container = widgets.Container(name=container_name, widgets=wdg_segment, layout="horizontal")
    # container.bind = btn.bind.__get__(container)
    container.value = container
    return container, table

In [6]:
# segmentation widget

segmentation_methods_params_table = []
segmentation_widgets = []


for segmentation in params["segmentation_params"]:
    container, table = get_container(segmentation, {segmentation:params["segmentation_params"][segmentation]})
    segmentation_widgets.append(container)
    segmentation_methods_params_table.append(table)


segmentation_methods_container = widgets.Container(widgets=segmentation_widgets, name="Choose_segmentation_method")
segmentation_methods_container.value = segmentation_methods_container
segmentation_methods_func = [Otsu, YAPIC, ChanVese]

@magicgui(call_button='Segment `input image`')
def segmentation_widget(
    input_image:ImageData,
    holder,
    nb_of_timestep_to_segment:int=10,
    all_timesteps:bool=False,
) -> LayerDataTuple:

    current_t = int(viewer.dims.point[0])
    original_shape = input_image.shape
    labels = []

    # create steps to display bit per bit
    if all_timesteps: 
        current_t = 0
        nb_of_timestep_to_segment = original_shape[0]
    step = max(1, min(nb_of_timestep_to_segment, 10))
    steps = np.linspace(current_t, max(1, nb_of_timestep_to_segment-step), int(nb_of_timestep_to_segment/step)).astype(int).tolist()
    print(steps)
    steps.append(nb_of_timestep_to_segment)

    for i in trange(len(segmentation_methods_container), label="Methods"):
        if segmentation_methods_container[i][0].value:
            mask = np.zeros(original_shape, dtype=bool)

            dict_params = segmentation_methods_params_table[i].to_dict()
            dict_params = dict_params[list(dict_params)[0]]

            for j in trange(len(steps)-1, label="steps"):
                t = steps[j]
                crop = input_image[t:t+steps[j+1]]
                mask_t = segmentation_methods_func[i](crop, dict_params).get_segmentation()
                mask[t:t+steps[j+1]] = mask_t

            print(steps[-1],nb_of_timestep_to_segment)
            crop = input_image[steps[-1]:nb_of_timestep_to_segment]
            mask_t = segmentation_methods_func[i](crop, dict_params).get_segmentation()
            mask[steps[-1]:nb_of_timestep_to_segment] = mask_t

            labels.append((mask, {'name':segmentation_methods_func[i].name}, "Labels"))

    return labels


viewer.window.add_dock_widget(segmentation_widget)

segmentation_widget.insert(1, segmentation_methods_container)
delattr(segmentation_widget, "holder")

In [7]:
@magicgui(
    call_button='get holes mask'
)
def inverse_mask(
    mask_label: LabelsData,
) -> LabelsData:
    holes_mask = np.zeros(mask_label.shape).astype(int)
    for i in trange(len(mask_label)-1, label="time"):
        holes_mask[i:i+1] = get_holes_mask(mask_label[i])
    holes_mask[len(holes_mask)-1:] = get_holes_mask(mask_label[len(holes_mask)-1])
    print(np.unique(holes_mask))
    return holes_mask
# add our new magicgui widget to the viewer

viewer.window.add_dock_widget(inverse_mask)

In [8]:
1+1

2

In [9]:
@magicgui(
    call_button='save tif image'
)
def save_tiff_wgt(
    mask_label: LabelsData,
    filename=pathlib.Path(params['output']['path']),
):
    print("gere")
    im_axes = params['output']['axes']
    save_tiff(filename, mask_label, im_axes)
# add our new magicgui widget to the viewer

viewer.window.add_dock_widget(save_tiff_wgt)

: 